In [ ]:
import numpy as np
import pandas as pd
from pypfopt import EfficientFrontier, risk_models, expected_returns
from vnstock3 import Vnstock
from pypfopt import EfficientCVaR, EfficientCDaR, expected_returns, HRPOpt
from pypfopt.efficient_frontier import efficient_semivariance

# Hàm lấy dữ liệu giá cổ phiếu từ vnstock3
def get_stock_data(symbols, start_date, end_date):
    all_data = []
    for symbol in symbols:
        stock = Vnstock().stock(symbol=symbol, source='VCI')
        try:
            stock_data = stock.quote.history(start=str(start_date), end=str(end_date))
            if stock_data is not None and not stock_data.empty:
                stock_data = stock_data[['time', 'close']].rename(columns={'close': symbol})
                stock_data['time'] = pd.to_datetime(stock_data['time'])
                all_data.append(stock_data.set_index('time'))
            else:
                print(f"Không có dữ liệu cho mã {symbol}.")
        except Exception as e:
            print(f"Lỗi khi lấy dữ liệu cho mã {symbol}: {e}")
    
    # Hợp nhất tất cả các DataFrame
    if all_data:
        combined_data = pd.concat(all_data, axis=1)
        # Xử lý giá trị thiếu bằng nội suy
        combined_data = combined_data.interpolate(method='linear', limit_direction='both')
        return combined_data
    else:
        return pd.DataFrame()

# Hàm mô hình Max Sharpe Ratio
def max_sharpe(data):
    mean_returns = expected_returns.mean_historical_return(data)
    cov_matrix = risk_models.sample_cov(data)

    ef = EfficientFrontier(mean_returns, cov_matrix)
    weights = ef.max_sharpe()  # Tối ưu hóa tỷ lệ Sharpe
    performance = ef.portfolio_performance(verbose=True)
    cleaned_weights = ef.clean_weights()

    print("\n=== Max Sharpe Portfolio ===")
    print("Trọng số danh mục:", cleaned_weights)
    print("Lợi nhuận kỳ vọng: {:.2%}".format(performance[0]))
    print("Rủi ro (Độ lệch chuẩn): {:.2%}".format(performance[1]))
    print("Tỷ lệ Sharpe: {:.2f}".format(performance[2]))
    return cleaned_weights, performance

# Hàm mô hình Min Volatility
def min_volatility(data):
    mean_returns = expected_returns.mean_historical_return(data)
    cov_matrix = risk_models.sample_cov(data)

    ef = EfficientFrontier(mean_returns, cov_matrix)
    weights = ef.min_volatility()  # Tối ưu hóa độ lệch chuẩn thấp nhất
    performance = ef.portfolio_performance(verbose=True)
    cleaned_weights = ef.clean_weights()

    print("\n=== Min Volatility Portfolio ===")
    print("Trọng số danh mục:", cleaned_weights)
    print("Lợi nhuận kỳ vọng: {:.2%}".format(performance[0]))
    print("Rủi ro (Độ lệch chuẩn): {:.2%}".format(performance[1]))
    print("Tỷ lệ Sharpe: {:.2f}".format(performance[2]))
    return cleaned_weights, performance
# Mô hình tối ưu hóa min_cdar
def min_cdar_model(symbols, start_date, end_date, beta=0.95):
    # Lấy dữ liệu giá cổ phiếu
    data = get_stock_data(symbols, start_date, end_date)
    if data.empty:
        print("Dữ liệu rỗng, không thể thực hiện tối ưu hóa.")
        return

    # Tính toán lợi nhuận kỳ vọng và lợi nhuận lịch sử
    mu = expected_returns.mean_historical_return(data)
    historical_rets = expected_returns.returns_from_prices(data).dropna()

    # Tạo đối tượng EfficientCDaR
    cdar_optimizer = EfficientCDaR(mu, historical_rets, beta=beta)

    # Tối ưu hóa danh mục theo CDaR
    weights = cdar_optimizer.min_cdar()
    performance = cdar_optimizer.portfolio_performance()

    # Hiển thị kết quả
    print("Trọng số tối ưu (min_cdar):", weights)
    print("Hiệu suất danh mục (min_cdar):")
    print(f"Lợi nhuận kỳ vọng: {performance[0]:.2%}")
    print(f"CDaR: {performance[1]:.2%}")

# Mô hình tối ưu hóa min_cvar
def min_cvar_model(symbols, start_date, end_date, beta=0.95):
    # Lấy dữ liệu giá cổ phiếu
    data = get_stock_data(symbols, start_date, end_date)
    if data.empty:
        print("Dữ liệu rỗng, không thể thực hiện tối ưu hóa.")
        return

    # Tính toán lợi nhuận kỳ vọng và lợi nhuận lịch sử
    mu = expected_returns.mean_historical_return(data)
    historical_rets = expected_returns.returns_from_prices(data).dropna()

    # Tạo đối tượng EfficientCVaR
    cvar_optimizer = EfficientCVaR(mu, historical_rets, beta=beta)

    # Tối ưu hóa danh mục theo CVaR
    weights = cvar_optimizer.min_cvar()
    performance = cvar_optimizer.portfolio_performance()

    # Hiển thị kết quả
    print("Trọng số tối ưu (min_cvar):", weights)
    print("Hiệu suất danh mục (min_cvar):")
    print(f"Lợi nhuận kỳ vọng: {performance[0]:.2%}")
    print(f"CVaR: {performance[1]:.2%}")
    return weights, performance

def Semivariance(symbols, start_date, end_date):
    # Tải dữ liệu từ vnstock3
    data = load_data_online(symbols, start_date, end_date)
    returns = expected_returns.returns_from_prices(data)

    # Semivariance
    ef_sv = efficient_semivariance(expected_returns.mean_historical_return(data), returns)
    ef_sv.min_semivariance()  # Sử dụng min_semivariance thay cho min_volatility
    print("Semivariance Portfolio:", ef_sv.clean_weights())
    performance_sv = ef_sv.portfolio_performance()
    print_performance(performance_sv, "Semivariance")

# Hàm mô hình HRP
def hrp_model(data):
    returns = data.pct_change().dropna(how="all")  # Tính lợi nhuận hàng ngày
    hrp = HRPOpt(returns)  # Khởi tạo mô hình HRP
    weights = hrp.optimize(linkage_method="single")  # Tối ưu hóa HRP
    performance = hrp.portfolio_performance()

    print("\n=== HRP Portfolio ===")
    print("Trọng số danh mục:", weights)
    print("Hiệu suất danh mục:")
    print(f"Lợi nhuận kỳ vọng: {performance[0]:.2%}")
    print(f"Độ lệch chuẩn (Rủi ro): {performance[1]:.2%}")
    print(f"Tỷ lệ Sharpe: {performance[2]:.2f}")
    return weights, performance
    
# Chạy thử mô hình
if __name__ == "__main__":
    # Danh sách mã cổ phiếu
    symbols = ['SSB', 'EIB', 'DSC', 'BCM', 'SGT', 'FPT','CHP','L10']
    start_date = '2020-01-01'
    end_date = '2023-01-01'

    # Lấy dữ liệu
    data = get_stock_data(symbols, start_date, end_date)

    if not data.empty:
        print("\n=== Dữ liệu giá cổ phiếu ===")
        print(data.head())

        # Mô hình Max Sharpe
        max_sharpe_weights, max_sharpe_performance = max_sharpe(data)

        # Mô hình Min Volatility
        min_vol_weights, min_vol_performance = min_volatility(data)
        
        # Chạy mô hình Min CDaR
        min_cdar_model(symbols, start_date, end_date, beta=0.95)
        
        # Chạy mô hình Min CVaR
        min_cvar_model(symbols, start_date, end_date, beta=0.95)
        # Chạy mô hình HRP
        hrp_weights, hrp_performance = hrp_model(data)
        # Chạy mô hình Semivariance
        Semivariance(symbols, start_date, end_date)

    else:
        print("Không có dữ liệu hợp lệ để chạy mô hình.")
